# Прогнозирование связей с помощью графовых нейронных сетей

In [1]:
import torch

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def set_seed():
    """
    Задает стартовое значение генератора псевдослучайных
    чисел для воспроизводимости.
    """
    torch.manual_seed(-1)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)

## Графовый автоэнкодер (VAE) и Вариационный графовый автоэнкодер (VGAE)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

set_seed()

transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, 
                      num_test=0.1, 
                      is_undirected=True, 
                      split_labels=True, 
                      add_negative_train_samples=False)
])

dataset = Planetoid('.', name='Cora', transform=transform)

train_data, val_data, test_data = dataset[0]

2024-01-17 17:12:26.936055: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from torch_geometric.nn import GCNConv, VGAE

class Encoder(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.conv1 = GCNConv(dim_in, 2 * dim_out)
        self.conv_mu = GCNConv(2 * dim_out, dim_out)
        self.conv_logstd = GCNConv(2 * dim_out, dim_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [4]:
model = VGAE(Encoder(dataset.num_features, 16)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

set_seed()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.pos_edge_label_index) + (
        1 / train_data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, 
                      data.pos_edge_label_index, 
                      data.neg_edge_label_index)

for epoch in range(301):
    loss = train()
    val_auc, val_ap = test(test_data)
    if epoch % 50 == 0:
        print(f'Эпоха {epoch:>2}:\n| Функция потерь: {loss:.4f} | '
              f'AUC-ROC на валид. наборе {val_auc:.4f} | '
              f'AP на валид. наборе: {val_ap:.4f}%')

test_auc, test_ap = test(test_data) 
print(f'AUC-ROC на тестовом наборе: {test_auc:.4f} | ' 
      f'AP на тестовом наборе {test_ap:.4f}')

Эпоха  0:
| Функция потерь: 3.5283 | AUC-ROC на валид. наборе 0.7082 | AP на валид. наборе: 0.7338%
Эпоха 50:
| Функция потерь: 1.3280 | AUC-ROC на валид. наборе 0.6889 | AP на валид. наборе: 0.7172%
Эпоха 100:
| Функция потерь: 1.2036 | AUC-ROC на валид. наборе 0.7246 | AP на валид. наборе: 0.7348%
Эпоха 150:
| Функция потерь: 1.0519 | AUC-ROC на валид. наборе 0.8147 | AP на валид. наборе: 0.8244%
Эпоха 200:
| Функция потерь: 0.9566 | AUC-ROC на валид. наборе 0.8625 | AP на валид. наборе: 0.8740%
Эпоха 250:
| Функция потерь: 0.9512 | AUC-ROC на валид. наборе 0.8679 | AP на валид. наборе: 0.8761%
Эпоха 300:
| Функция потерь: 0.9287 | AUC-ROC на валид. наборе 0.8751 | AP на валид. наборе: 0.8795%
AUC-ROC на тестовом наборе: 0.8751 | AP на тестовом наборе 0.8795


In [5]:
z = model.encode(test_data.x, test_data.edge_index) 
Ahat = torch.sigmoid(z @ z.T)
Ahat

tensor([[0.8203, 0.6496, 0.7584,  ..., 0.5449, 0.8411, 0.8268],
        [0.6496, 0.8801, 0.8621,  ..., 0.4629, 0.8050, 0.7793],
        [0.7584, 0.8621, 0.8831,  ..., 0.4940, 0.8652, 0.8438],
        ...,
        [0.5449, 0.4629, 0.4940,  ..., 0.5227, 0.5337, 0.5298],
        [0.8411, 0.8050, 0.8652,  ..., 0.5337, 0.9043, 0.8854],
        [0.8268, 0.7793, 0.8438,  ..., 0.5298, 0.8854, 0.8671]],
       grad_fn=<SigmoidBackward0>)

## SEAL

In [6]:
import numpy as np
from sklearn.metrics import (roc_auc_score, 
                             average_precision_score)
from scipy.sparse.csgraph import shortest_path

import torch.nn.functional as F
from torch.nn import (Conv1d, 
                      MaxPool1d, 
                      Linear,
                      Dropout,
                      BCEWithLogitsLoss)

from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import aggr
from torch_geometric.utils import (k_hop_subgraph, 
                                   to_scipy_sparse_matrix)

In [7]:
set_seed()

# загружаем набор данных Cora
transform = RandomLinkSplit(num_val=0.05, 
                            num_test=0.1, 
                            is_undirected=True, 
                            split_labels=True)
dataset = Planetoid('.', name='Cora', transform=transform)
train_data, val_data, test_data = dataset[0]
train_data

Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[4488], pos_edge_label_index=[2, 4488], neg_edge_label=[4488], neg_edge_label_index=[2, 4488])

In [8]:
def seal_processing(dataset, edge_label_index, y):
    data_list = []

    for src, dst in edge_label_index.t().tolist():
        sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
            [src, dst], 2, dataset.edge_index, relabel_nodes=True
        )
        src, dst = mapping.tolist()

        # выполняем фильтрацию ребер в подграфе
        mask1 = (sub_edge_index[0] != src) | (sub_edge_index[1] != dst)
        mask2 = (sub_edge_index[0] != dst) | (sub_edge_index[1] != src)
        sub_edge_index = sub_edge_index[:, mask1 & mask2]

        # разметка узлов на основе двойного радиуса (DRNL)
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(
            sub_edge_index, num_nodes=sub_nodes.size(0)).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        # вычисляем расстояние между каждым узлом и целевым узлом-источником
        d_src = shortest_path(adj_wo_dst, directed=False, 
                              unweighted=True, indices=src)
        d_src = np.insert(d_src, dst, 0, axis=0)
        d_src = torch.from_numpy(d_src)

        # вычисляем расстояние между каждым узлом и целевым узлом-отправлением
        d_dst = shortest_path(adj_wo_src, directed=False, 
                              unweighted=True, indices=dst-1)
        d_dst = np.insert(d_dst, src, 0, axis=0)
        d_dst = torch.from_numpy(d_dst)

        # вычисляем метку z для каждого узла
        dist = d_src + d_dst
        z = 1 + torch.min(d_src, d_dst) + dist // 2 * (dist // 2 + dist % 2 - 1)
        z[src], z[dst], z[torch.isnan(z)] = 1., 1., 0.
        z = z.to(torch.long)
        
        # конкатенируем признаки узлов и one-hot закодированные метки
        # узлов (с фиксированным количеством классов)
        node_labels = F.one_hot(z, num_classes=200).to(torch.float)
        node_emb = dataset.x[sub_nodes]
        node_x = torch.cat([node_emb, node_labels], dim=1)

        # создаем объект Data
        data = Data(x=node_x, z=z, edge_index=sub_edge_index, y=y)
        data_list.append(data)

    return data_list

In [9]:
# извлечение охватывающих подграфов
train_pos_data_list = seal_processing(
    train_data, train_data.pos_edge_label_index, 1)
train_neg_data_list = seal_processing(
    train_data, train_data.neg_edge_label_index, 0)

val_pos_data_list = seal_processing(
    val_data, val_data.pos_edge_label_index, 1)
val_neg_data_list = seal_processing(
    val_data, val_data.neg_edge_label_index, 0)

test_pos_data_list = seal_processing(
    test_data, test_data.pos_edge_label_index, 1)
test_neg_data_list = seal_processing(
    test_data, test_data.neg_edge_label_index, 0)

In [10]:
train_dataset = train_pos_data_list + train_neg_data_list
val_dataset = val_pos_data_list + val_neg_data_list
test_dataset = test_pos_data_list + test_neg_data_list

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [11]:
class DGCNN(torch.nn.Module):
    def __init__(self, dim_in, k=30):
        super().__init__()

        # GCN-слои
        self.gcn1 = GCNConv(dim_in, 32)
        self.gcn2 = GCNConv(32, 32)
        self.gcn3 = GCNConv(32, 32)
        self.gcn4 = GCNConv(32, 1)

        # глобальный сортировочный пулинг
        self.global_pool = aggr.SortAggregation(k=k)

        # слои свертки
        self.conv1 = Conv1d(1, 16, 97, 97)
        self.conv2 = Conv1d(16, 32, 5, 1)
        self.maxpool = MaxPool1d(2, 2)

        # плотные слои
        self.linear1 = Linear(352, 128)
        self.dropout = Dropout(0.5)
        self.linear2 = Linear(128, 1)

    def forward(self, x, edge_index, batch):
        # 1. слои графовой свертки
        h1 = self.gcn1(x, edge_index).tanh()
        h2 = self.gcn2(h1, edge_index).tanh()
        h3 = self.gcn3(h2, edge_index).tanh()
        h4 = self.gcn4(h3, edge_index).tanh()
        h = torch.cat([h1, h2, h3, h4], dim=-1)

        # 2. глобальный сортировочный пулинг
        h = self.global_pool(h, batch)

        # 3. традиционные слои свертки и плотные слои
        h = h.view(h.size(0), 1, h.size(-1))
        h = self.conv1(h).relu()
        h = self.maxpool(h)
        h = self.conv2(h).relu()
        h = h.view(h.size(0), -1)
        h = self.linear1(h).relu()
        h = self.dropout(h)
        h = self.linear2(h).sigmoid()

        return h

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(train_dataset[0].num_features).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = BCEWithLogitsLoss()

set_seed()

def train():
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_dataset)

@torch.no_grad()
def test(loader):
    model.eval()
    y_pred, y_true = [], []

    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        y_pred.append(out.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    auc = roc_auc_score(torch.cat(y_true), torch.cat(y_pred))
    ap = average_precision_score(torch.cat(y_true), torch.cat(y_pred))

    return auc, ap

for epoch in range(31):
    loss = train()
    val_auc, val_ap = test(val_loader)
    print(f'Эпоха {epoch:>2}:\n| Функция потерь: {loss:.4f} | '
          f'AUC-ROC на валид. наборе {val_auc:.4f} | '
          f'AP на валид. наборе: {val_ap:.4f}%')

test_auc, test_ap = test(test_loader)
print(f'AUC-ROC на тестовом наборе: {test_auc:.4f} | ' 
      f'AP на тестовом наборе {test_ap:.4f}')

Эпоха  0:
| Функция потерь: 0.6996 | AUC-ROC на валид. наборе 0.7970 | AP на валид. наборе: 0.8178%
Эпоха  1:
| Функция потерь: 0.6240 | AUC-ROC на валид. наборе 0.8438 | AP на валид. наборе: 0.8706%
Эпоха  2:
| Функция потерь: 0.5889 | AUC-ROC на валид. наборе 0.8526 | AP на валид. наборе: 0.8818%
Эпоха  3:
| Функция потерь: 0.5811 | AUC-ROC на валид. наборе 0.8637 | AP на валид. наборе: 0.8908%
Эпоха  4:
| Функция потерь: 0.5765 | AUC-ROC на валид. наборе 0.8681 | AP на валид. наборе: 0.8956%
Эпоха  5:
| Функция потерь: 0.5731 | AUC-ROC на валид. наборе 0.8710 | AP на валид. наборе: 0.8971%
Эпоха  6:
| Функция потерь: 0.5697 | AUC-ROC на валид. наборе 0.8718 | AP на валид. наборе: 0.8936%
Эпоха  7:
| Функция потерь: 0.5672 | AUC-ROC на валид. наборе 0.8729 | AP на валид. наборе: 0.8937%
Эпоха  8:
| Функция потерь: 0.5646 | AUC-ROC на валид. наборе 0.8799 | AP на валид. наборе: 0.8921%
Эпоха  9:
| Функция потерь: 0.5601 | AUC-ROC на валид. наборе 0.8832 | AP на валид. наборе: 0.8936%
